<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-All-Dependencies" data-toc-modified-id="Import-All-Dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import All Dependencies</a></span></li><li><span><a href="#Set-Paths-to-Model-and-Index-Files" data-toc-modified-id="Set-Paths-to-Model-and-Index-Files-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set Paths to Model and Index Files</a></span></li><li><span><a href="#Compute-Number-Of-Syllables-that-Explain-99%-of-the-Data's-Variance" data-toc-modified-id="Compute-Number-Of-Syllables-that-Explain-99%-of-the-Data's-Variance-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Compute Number Of Syllables that Explain 99% of the Data's Variance</a></span></li><li><span><a href="#Compute-Syllable-Statistics" data-toc-modified-id="Compute-Syllable-Statistics-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Compute Syllable Statistics</a></span></li><li><span><a href="#Compute-Scalar-Means-(By-Group)-For-Each-Syllable" data-toc-modified-id="Compute-Scalar-Means-(By-Group)-For-Each-Syllable-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Compute Scalar Means (By Group) For Each Syllable</a></span></li><li><span><a href="#Plot-Mean-Statistics-Sorted-By-Group-Difference" data-toc-modified-id="Plot-Mean-Statistics-Sorted-By-Group-Difference-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Plot Mean Statistics Sorted By Group Difference</a></span></li><li><span><a href="#Compute-Scalar-Means-(By-Session)-For-Each-Syllable" data-toc-modified-id="Compute-Scalar-Means-(By-Session)-For-Each-Syllable-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Compute Scalar Means (By Session) For Each Syllable</a></span></li><li><span><a href="#Compute-Syllable-Distance-Matrices" data-toc-modified-id="Compute-Syllable-Distance-Matrices-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Compute Syllable Distance Matrices</a></span></li><li><span><a href="#Plot-Syllable-Distance-Dendrogram" data-toc-modified-id="Plot-Syllable-Distance-Dendrogram-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Plot Syllable Distance Dendrogram</a></span></li><li><span><a href="#Plot-Behavioral-Distance-Confusion-Matrix" data-toc-modified-id="Plot-Behavioral-Distance-Confusion-Matrix-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Plot Behavioral Distance Confusion Matrix</a></span></li><li><span><a href="#Compute-Syllable-Transition-Matrices" data-toc-modified-id="Compute-Syllable-Transition-Matrices-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Compute Syllable Transition Matrices</a></span></li><li><span><a href="#Plot-Transition-Matrices" data-toc-modified-id="Plot-Transition-Matrices-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Plot Transition Matrices</a></span></li><li><span><a href="#Plot-Computed-Transition-Graphs" data-toc-modified-id="Plot-Computed-Transition-Graphs-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Plot Computed Transition Graphs</a></span></li><li><span><a href="#Hypothesis-Testing" data-toc-modified-id="Hypothesis-Testing-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Hypothesis Testing</a></span><ul class="toc-item"><li><span><a href="#Mann-Whitney-Hypothesis-Test-(2-Groups)" data-toc-modified-id="Mann-Whitney-Hypothesis-Test-(2-Groups)-14.1"><span class="toc-item-num">14.1&nbsp;&nbsp;</span>Mann-Whitney Hypothesis Test (2-Groups)</a></span></li><li><span><a href="#Kruskal-Wallis-and-Dunn's-Hypothesis-Tests" data-toc-modified-id="Kruskal-Wallis-and-Dunn's-Hypothesis-Tests-14.2"><span class="toc-item-num">14.2&nbsp;&nbsp;</span>Kruskal-Wallis and Dunn's Hypothesis Tests</a></span></li><li><span><a href="#Perform-Bootstrapped-Z-Test" data-toc-modified-id="Perform-Bootstrapped-Z-Test-14.3"><span class="toc-item-num">14.3&nbsp;&nbsp;</span>Perform Bootstrapped Z-Test</a></span></li><li><span><a href="#Perform-T-Test" data-toc-modified-id="Perform-T-Test-14.4"><span class="toc-item-num">14.4&nbsp;&nbsp;</span>Perform T-Test</a></span></li></ul></li><li><span><a href="#Classifiers" data-toc-modified-id="Classifiers-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Classifiers</a></span></li><li><span><a href="#Embedding" data-toc-modified-id="Embedding-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Embedding</a></span><ul class="toc-item"><li><span><a href="#LDA" data-toc-modified-id="LDA-16.1"><span class="toc-item-num">16.1&nbsp;&nbsp;</span>LDA</a></span></li></ul></li></ul></div>

## Import All Dependencies

In [ ]:
%matplotlib inline
# Syllable Statistics Dependencies
import matplotlib.pyplot as plt
from moseq2_viz.util import parse_index
from moseq2_viz.viz import plot_syll_stats_with_sem
from moseq2_viz.scalars.util import scalars_to_dataframe
from moseq2_viz.model.stat import (bootstrap_me, get_session_mean_df, run_kruskal,
                                   run_ztest, run_ttest, run_mann_whitney_test,
                                   train_classifier, run_multiple_comparisons_test,
                                   run_LDA)
from moseq2_viz.model.util import (compute_behavioral_statistics, 
                                   sort_syllables_by_stat, get_syllable_usages,
                                   compute_syllable_explained_variance,
                                   sort_syllables_by_stat_difference)

# Behavioral Distance Dependencies
from scipy.cluster.hierarchy import linkage, dendrogram
from moseq2_viz.model.dist import get_behavioral_distance

# Transition Matrices Compute/Graphing Dependencies
from moseq2_viz.model.util import parse_model_results, relabel_by_usage
from moseq2_viz.model.trans_graph import get_trans_graph_groups, get_group_trans_mats, graph_transition_matrix

# Hypothesis Testing Dependencies
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import preprocessing
from statsmodels.stats.multitest import multipletests

## Set Paths to Model and Index Files

In [ ]:
index_path = './moseq2-index.yaml'
model_path = './saline-amphetamine/model.p'
_, sorted_index = parse_index(index_path)

## Compute Number Of Syllables that Explain 99% of the Data's Variance

In [ ]:
model_data = parse_model_results(model_path)
model_data['labels'] = relabel_by_usage(model_data['labels'], count='usage')[0]
max_syllable = compute_syllable_explained_variance(model_data, n_explained=99)

## Compute Syllable Statistics

In [ ]:
_, sorted_index = parse_index(index_path)

# compute session scalar data
scalar_df = scalars_to_dataframe(sorted_index, model_path=model_path)

# compute syllable usage and scalar statistics
df = compute_behavioral_statistics(scalar_df, count='usage', groupby=['group', 'uuid'], usage_normalization=True)

## Compute Scalar Means (By Group) For Each Syllable

In [ ]:
stat = 'usage'

unique_groups = df.group.unique()

plot_syll_stats_with_sem(df, 
                         stat='usage',
                         ordering='stat',
                         max_sylls=max_syllable,
                         groups=unique_groups,
                         ctrl_group=None,
                         exp_group=None,
                         colors=None,
                         join=True,
                         figsize=(15,7))

## Plot Mean Statistics Sorted By Group Difference

In [ ]:
# statistic to plot
stat = 'usage'

# groups to get difference
group1 = 'Saline'
group2 = 'Amphetamine'

unique_groups = df.group.unique()

plot_syll_stats_with_sem(df, 
                         stat='usage', # choose any of the df columns
                         ordering='diff',
                         max_sylls=max_syllable,
                         groups=unique_groups,
                         ctrl_group=group1,
                         exp_group=group2,
                         colors=None,
                         join=True,
                         figsize=(15,7))

## Compute Scalar Means (By Session) For Each Syllable

In [ ]:
%matplotlib inline

# you may switch 'uuid' with 'SessionName' or 'SubjectName' if they are all unique
session_mean_df = df.groupby(['syllable', 'uuid']).mean() 

stat = 'usage'

fig, ax = plt.subplots(figsize=(15,7))
session_mean_df[stat].unstack().plot(style='o-', ax=ax, legend=False)

## Compute Syllable Distance Matrices

In [ ]:
# Get Pairwise distances
X = get_behavioral_distance(sorted_index,
                            model_path,
                            max_syllable=max_syllable,
                            distances=['ar[init]'])['ar[init]']

## Plot Syllable Distance Dendrogram

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(1, 1, figsize=(15, 7))

# Compute linkage matrix with respect to a selected precomputed metric
Z = linkage(np.nan_to_num(X), 'complete')
results = dendrogram(Z, distance_sort=False, get_leaves=True, ax=ax, color_threshold=0)
similarity_ordering = results['leaves']

## Plot Behavioral Distance Confusion Matrix

In [ ]:
fig, ax = plt.subplots(1)
h = ax.imshow(X, aspect='equal', cmap='cubehelix')
ax.set_title('Behavioral distance')
ax.set_xlabel('Syllable j')
ax.set_ylabel('Syllable i')
plt.colorbar(h, ax=ax)
plt.show()

## Compute Syllable Transition Matrices

In [ ]:
# load your model
model_data = parse_model_results(model_path)

# select a transition matrix normalization method
normalize = 'bigram' # other options: 'columns', 'rows'

# Get labels and relabel them by usage sorting
labels = model_data['labels']
labels = relabel_by_usage(labels, count='usage')[0]

# Get modeled session uuids to compute group-mean transition graph for
group, label_group, _ = get_trans_graph_groups(model_data, sorted_index)

# compute transition matrices and usages for each group
trans_mats, usages = get_group_trans_mats(labels, label_group, group, max_syllable, normalize=normalize)

## Plot Transition Matrices

In [ ]:
fig, ax = plt.subplots(1, len(group), figsize=(12, 8), sharex=False, sharey=True)

for i, g in enumerate(group):
    h = ax[i].imshow(trans_mats[i][:max_syllable,:max_syllable], cmap='magma')
    plt.colorbar(h, ax=ax[i], fraction=0.046, pad=0.04)
    ax[i].set_title(f'{g}: Bigram Transition Probabilities')

## Plot Computed Transition Graphs

In [ ]:
fig, _, _ = graph_transition_matrix(trans_mats,
                                    layout='spring', # or: 'circular', 'spectral'
                                    usages=usages.copy(),
                                    groups=group,
                                    arrows=True,
                                    headless=True)

## Hypothesis Testing

### Mann-Whitney Hypothesis Test (2-Groups)

In [ ]:
statistic = 'usage'
group1 = 'Saline'
group2 = 'Amphetamine'

df_mw_real, exclude_sylls = run_mann_whitney_test(df,
                                                  group1,
                                                  group2,
                                                  statistic=statistic,
                                                  max_syllable=max_syllable,
                                                  THRESH=0.05,
                                                  MC_METHOD='fdr_bh')

### Kruskal-Wallis and Dunn's Hypothesis Tests

In [ ]:
# Set KW Constants
seed = 42
THRESH = 0.05
n_perm = 10000
statistic = 'usage'
MC_METHOD = "fdr_bh"

kruskal_results_df, dunn_results_df, intersect_sig_syllables = run_kruskal(df, 
                                                                           statistic=statistic,
                                                                           max_syllable=max_syllable,
                                                                           n_perm=n_perm,
                                                                           seed=seed,
                                                                           THRESH=THRESH,
                                                                           MC_METHOD=MC_METHOD)

### Perform Bootstrapped Z-Test

In [ ]:
pvals_ztest_boots, z_syllables_to_include = run_ztest(df, group1, group2, statistic, max_syllable=max_syllable)

### Perform T-Test

In [ ]:
pvals_ttest, t_syllables_to_include = run_ttest(df, group1, group2, statistic, max_syllable=max_syllable)
t_syllables_to_include